# Generate AIS reception and gap events datasets

This notebook is a wrapper file for producing a complete set of results and inputs for Welch et al. (2021). It contains code for the following: 

1. Generate raw AIS gap events greater than 12 hours
2. Generate monthly AIS reception maps
3. Detect suspected AIS disabling events

## Setup

### Packages

In [1]:
# Modules
import os
import pandas as pd
import numpy as np
import pandas_gbq
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import time
from google.cloud import bigquery
from jinja2 import Template

import pyseas
import pyseas.maps
import pyseas.maps.rasters
import pyseas.styles
import pyseas.cm

# project specific functions
import utils 

%load_ext autoreload
%autoreload 2

# BigQuery client
client = bigquery.Client()

/opt/miniconda3/envs/gfw/lib/python3.8/site-packages/h3/unstable/__init__.py:4: UserWarning: Modules under `h3.unstable` are experimental, and may change at any time.
  warnings.warn(
/opt/miniconda3/envs/gfw/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/miniconda3/envs/gfw/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" 

### Inputs & Parameters

In [44]:
# Input BQ datasets/tables
gfw_research = 'gfw_research'
gfw_research_precursors = 'gfw_research_precursors'
destination_dataset = 'scratch_tyler'

pipeline_version = 'v20201001'
pipeline_table = 'pipe_{}'.format(pipeline_version)
segs_table = 'pipe_{}_segs'.format(pipeline_version)
vi_version = 'v20210706'

# Output tables version
output_version = 'v20210721'
create_tables = True

# Date range
start_date = date(2017,1, 1)
end_date = date(2017,1, 4)

# Min gap hours
min_gap_hours = 6

Generate list of dates to produce for analysis.

In [45]:
# Generate list of dates to run
dates_to_run = utils.daterange(start_date, end_date)
tp = []
for dt in dates_to_run:
    tp.append(dt.strftime("%Y-%m-%d"))

## AIS Gaps dataset

Generate a dataset of AIS gaps for time range. This involves running the following query sequence (queries in the `gaps` subdirectory):
1. AIS off events: `ais_off_on_events.sql.j2` with `event` parameter set to `'off'`
2. AIS on events: `ais_off_on_events.sql.j2` with `event` parameter set to `'on'`
3. AIS gap events: Stitch off and on events together into gap events using `ais_gap_events.sql.j2`

### Create tables

First, create empty tables for all three tables.

In [46]:
# Destination tables
off_events_table = 'ais_off_events_{}'.format(output_version)
on_events_table = 'ais_on_events_{}'.format(output_version)
gap_events_table = 'ais_gap_events_{}'.format(output_version)

Create tables for off/on events.

In [47]:
if create_tables:
    # Off events
    utils.make_bq_partitioned_table(destination_dataset, off_events_table)
    # On events
    utils.make_bq_partitioned_table(destination_dataset, on_events_table)

### Off events

Generate off events

In [48]:
# Store commands
cmds = []
for t in tp:
    cmd = utils.make_ais_events_table(pipeline_table="{}.{}".format("gfw_research", pipeline_table),
                                segs_table="{}.{}".format("gfw_research", segs_table),
                                event_type='off',
                                date = t,
                                min_gap_hours = min_gap_hours, 
                                precursors_dataset=destination_dataset,
                                destination_table=off_events_table)
    cmds.append(cmd)

In [27]:
# test query
# test_cmd = cmds[0].split('|')[0]
# os.system(test_cmd)
# os.system(cmds[0])

0

In [49]:
# Run queries
utils.execute_commands_in_parallel(commands=cmds)

['jinja2 gaps/ais_off_on_events.sql.j2     -D pipeline_table="gfw_research.pipe_v20201001"     -D segs_table="gfw_research.pipe_v20201001_segs"     -D event="off"     -D date="2017-01-01"     -D min_gap_length=6     |     bq query --replace     --destination_table=scratch_tyler.ais_off_events_v20210721\\$20170101    --allow_large_results --use_legacy_sql=false --max_rows=0\n    ', 'jinja2 gaps/ais_off_on_events.sql.j2     -D pipeline_table="gfw_research.pipe_v20201001"     -D segs_table="gfw_research.pipe_v20201001_segs"     -D event="off"     -D date="2017-01-02"     -D min_gap_length=6     |     bq query --replace     --destination_table=scratch_tyler.ais_off_events_v20210721\\$20170102    --allow_large_results --use_legacy_sql=false --max_rows=0\n    ', 'jinja2 gaps/ais_off_on_events.sql.j2     -D pipeline_table="gfw_research.pipe_v20201001"     -D segs_table="gfw_research.pipe_v20201001_segs"     -D event="off"     -D date="2017-01-03"     -D min_gap_length=6     |     bq query --r

### On events

Generate on events.

In [50]:
# Store commands
on_cmds = []
for t in tp:
    cmd = utils.make_ais_events_table(pipeline_table="{}.{}".format("gfw_research", pipeline_table),
                                segs_table="{}.{}".format("gfw_research", segs_table),
                                event_type='on',
                                date = t,
                                min_gap_hours = min_gap_hours, 
                                precursors_dataset=destination_dataset,
                                destination_table=on_events_table)
    on_cmds.append(cmd)

In [51]:
# test query
# test_cmd = on_cmds[0].split('|')[0]
# os.system(test_cmd)

In [52]:
# Run queries
utils.execute_commands_in_parallel(commands=on_cmds)

['jinja2 gaps/ais_off_on_events.sql.j2     -D pipeline_table="gfw_research.pipe_v20201001"     -D segs_table="gfw_research.pipe_v20201001_segs"     -D event="on"     -D date="2017-01-01"     -D min_gap_length=6     |     bq query --replace     --destination_table=scratch_tyler.ais_on_events_v20210721\\$20170101    --allow_large_results --use_legacy_sql=false --max_rows=0\n    ', 'jinja2 gaps/ais_off_on_events.sql.j2     -D pipeline_table="gfw_research.pipe_v20201001"     -D segs_table="gfw_research.pipe_v20201001_segs"     -D event="on"     -D date="2017-01-02"     -D min_gap_length=6     |     bq query --replace     --destination_table=scratch_tyler.ais_on_events_v20210721\\$20170102    --allow_large_results --use_legacy_sql=false --max_rows=0\n    ', 'jinja2 gaps/ais_off_on_events.sql.j2     -D pipeline_table="gfw_research.pipe_v20201001"     -D segs_table="gfw_research.pipe_v20201001_segs"     -D event="on"     -D date="2017-01-03"     -D min_gap_length=6     |     bq query --replac

### Gap events

Combine off and on events into gap events.

Create gap events table, partitioning on the `gap_start` field.

In [69]:
if create_tables:
    gap_tbl_cmd = "bq mk --schema=gaps/ais_gap_events.json \
    --time_partitioning_field=gap_start \
    --time_partitioning_type=DAY {}.{}".format(destination_dataset, 
                                               gap_events_table)
    os.system(gap_tbl_cmd)

/opt/miniconda3/envs/gfw/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/miniconda3/envs/gfw/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about servic

In [70]:
latest_date = tp[-1]
gap_cmd = utils.make_ais_gap_events_table(off_events_table = off_events_table,
                                on_events_table = on_events_table,
                                date = latest_date,
                                precursors_dataset = destination_dataset,
                                destination_dataset = destination_dataset,
                                destination_table = gap_events_table)

In [55]:
# test query
# test_cmd = gap_cmd.split('|')[0]
# os.system(test_cmd)
# os.system(test_cmd)

In [71]:
# Run command
os.system(gap_cmd)

0

In [72]:
# Update schema
gap_schema_cmd = "bq update --schema=gaps/ais_gap_events.json {}.{}".format(destination_dataset, gap_events_table)
os.system(gap_schema_cmd)

0

## AIS Interpolation

The next step is to generate tables of interpolated vessel positions. These tables are used subsequently for the following:
- AIS reception
- Time lost to gaps

> The original reception quality method used a slightly different interpolation [query](https://github.com/GlobalFishingWatch/ais-gaps-and-reception/blob/master/data-production/hourly_interpoloation_v20191120.sql.j2)/table (`gfw_research_precursors.ais_positions_byssvid_hourly_v20191118`) than the [query](https://github.com/GlobalFishingWatch/ais-gaps-and-reception/blob/master/data-production/pipe-interpolation/hourly_interpoloation_v20201027.sql.j2) used to estimate time lost to gaps. These approaches have been combined/streamlined into the `interpolation/hourly_interpolation_byseg.sql.j2` in this repo.

### Create tables

First create empty date partitioned tables to store interpolated positions.

In [72]:
# Destination tables
ais_positions_hourly = 'ais_positions_byssvid_hourly_{}'.format(output_version)
ais_positions_hourly_fishing = 'ais_positions_byssvid_hourly_fishing_{}'.format(output_version)
gap_positions_hourly = 'gap_positions_byssvid_hourly_{}'.format(output_version)
loitering_positions_hourly = 'loitering_positions_byssvid_hourly_{}'.format(output_version)

# By seg_id
ais_positions_hourly = 'ais_positions_byseg_hourly_{}'.format(output_version)

In [74]:
if create_tables:
    # all positions hourly
    utils.make_bq_partitioned_table(destination_dataset, ais_positions_hourly)
    # fishing vessel positions hourly
    utils.make_bq_partitioned_table(destination_dataset, ais_positions_hourly_fishing)
    # gap positions hourly
    utils.make_bq_partitioned_table(destination_dataset, gap_positions_hourly)

### Interpolate all vessel positions

Interpolate positions for all vessels.

In [87]:
# Store commands
int_cmds = []
for t in tp:
    cmd = utils.make_hourly_interpolation_table(date = t,
                                                destination_dataset = destination_dataset,
                                                destination_table = ais_positions_hourly)
    int_cmds.append(cmd)

In [89]:
utils.execute_commands_in_parallel(int_cmds)

['jinja2 interpolation/hourly_interpolation_byseg.sql.j2           -D YYYY_MM_DD="2017-01-01"        |         bq query --replace         --destination_table=scratch_tyler.ais_positions_byseg_hourly_v20210429\\$20170101         --allow_large_results --use_legacy_sql=false ', 'jinja2 interpolation/hourly_interpolation_byseg.sql.j2           -D YYYY_MM_DD="2017-01-02"        |         bq query --replace         --destination_table=scratch_tyler.ais_positions_byseg_hourly_v20210429\\$20170102         --allow_large_results --use_legacy_sql=false ', 'jinja2 interpolation/hourly_interpolation_byseg.sql.j2           -D YYYY_MM_DD="2017-01-03"        |         bq query --replace         --destination_table=scratch_tyler.ais_positions_byseg_hourly_v20210429\\$20170103         --allow_large_results --use_legacy_sql=false ', 'jinja2 interpolation/hourly_interpolation_byseg.sql.j2           -D YYYY_MM_DD="2017-01-04"        |         bq query --replace         --destination_table=scratch_tyler.ais

### Interpolate fishing vessel positions
Interpolate positions for fishing vessels, including both `nnet_score` and `night_loitering` for determining when `squid_jiggers` are fishing.

TODO: Update this to the same logic as for all vessels.

In [10]:
# Store commands
# int_fishing_cmds = []
# for t in tp:
#     cmd = utils.make_hourly_fishing_interpolation_table(date = t,
#                                                 destination_dataset = destination_dataset,
#                                                 destination_table = ais_positions_hourly_fishing)
#     int_fishing_cmds.append(cmd)

In [13]:
# utils.execute_commands_in_parallel(int_fishing_cmds)

['jinja2 interpolation/hourly_fishing_interpolation.sql.j2           -D YYYY_MM_DD="2017-01-01"        |         bq query --replace         --destination_table=scratch_tyler.ais_positions_byssvid_hourly_fishing_v20210429\\$20170101         --allow_large_results --use_legacy_sql=false ', 'jinja2 interpolation/hourly_fishing_interpolation.sql.j2           -D YYYY_MM_DD="2017-01-02"        |         bq query --replace         --destination_table=scratch_tyler.ais_positions_byssvid_hourly_fishing_v20210429\\$20170102         --allow_large_results --use_legacy_sql=false ', 'jinja2 interpolation/hourly_fishing_interpolation.sql.j2           -D YYYY_MM_DD="2017-01-03"        |         bq query --replace         --destination_table=scratch_tyler.ais_positions_byssvid_hourly_fishing_v20210429\\$20170103         --allow_large_results --use_legacy_sql=false ', 'jinja2 interpolation/hourly_fishing_interpolation.sql.j2           -D YYYY_MM_DD="2017-01-04"        |         bq query --replace        

### Interpolate positions during AIS gap events

> **Note:** Interpolating positions between gap events was originally done using the `raw_gaps_vYYYYMMDD` table, which included the gaps with additional parameters applied to them - e.g. `pos_x_hours_before`. Need to produce a version of this table or interpolate the gap events as is.

## AIS Reception Quality

Model AIS satellite reception quality to identify regions where AIS gap events are more/less suspicious. This is produced using the following process:

**1. Calculate measured reception** - Calculates measured reception quality by AIS Class as the average number of positions received by a vessel in a day per one-degree grid cell

**2. Interpolate reception** - To produce global maps of reception quality (e.g. not just in cells with AIS data) use a smoothing function to interpolate reception quality. 

### Create tables

In [15]:
sat_reception_measured = 'sat_reception_measured_one_degree_{}'.format(output_version)
sat_reception_smoothed = 'sat_reception_smoothed_one_degree_{}'.format(output_version)

In [17]:
if create_tables:
    # measured reception quality
    utils.make_bq_partitioned_table(destination_dataset, sat_reception_measured)
    # smoothed reception quality
    utils.make_bq_partitioned_table(destination_dataset, sat_reception_smoothed)

### Measured reception quality

In [90]:
# Make list of month start dates for reception quality
reception_dates = pd.date_range(start_date, end_date, freq='1M') - pd.offsets.MonthBegin(1)

In [91]:
# Generate commands
mr_cmds = []
for r in reception_dates:
    print(str(r.date()))
    cmd = utils.make_reception_measured_table(destination_table = sat_reception_measured, 
                                        destination_dataset = destination_dataset,
                                        start_date = r, 
                                        vi_version = vi_version, 
                                        segs_table="{}.{}".format("gfw_research", segs_table),
                                        output_version = output_version)

    mr_cmds.append(cmd)

2017-01-01


In [93]:
utils.execute_commands_in_parallel(mr_cmds)

['jinja2 reception/reception_measured.sql.j2           -D start_date="2017-01-01"        -D end_date="2017-02-01"        -D vi_version="v20210301"        -D segs_table="gfw_research.pipe_v20201001_segs"        -D destination_dataset="scratch_tyler"        -D output_version="v20210429"        |         bq query --replace         --destination_table=scratch_tyler.sat_reception_measured_one_degree_v20210429\\$20170101          --allow_large_results --use_legacy_sql=false']


### Smoothed reception quality

Next, interpolate the measured reception quality using a radial basis function. 

In [117]:
for r in reception_dates:
    print(str(r.date()))
    utils.make_smooth_reception_table(start_date = r,
                                      reception_measured_table = sat_reception_measured,
                                      destination_dataset = destination_dataset,
                                      destination_table = sat_reception_smoothed)

/opt/miniconda3/envs/gfw/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/miniconda3/envs/gfw/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about servic

2017-01-01
Querying reception for 2017-01-01 00:00:00
Interpolating reception for 2017-01-01 00:00:00
Loaded 129600 rows for 2017-01-01 00:00:00 into sat_reception_smoothed_one_degree_v20210429.


#### Plot reception quality

In [122]:
for r in reception_dates:
    print(str(r.date()))
    utils.plot_reception_quality(reception_start_date = r,
                           destination_dataset = destination_dataset,
                           reception_smoothed_table = sat_reception_smoothed)

2017-01-01


AttributeError: module 'utils' has no attribute 'plot_reception_quality'